# 4 Non-Parametric Tests

Learn about the assumptions made by parametric hypothesis tests and see how simulation-based and rank-based non-parametric tests can be used when those assumptions aren't met.

# Common assumptions of hypothesis tests

Hypothesis tests make assumptions about the dataset that they are testing, and the conclusions you draw from the test results are only valid if those assumptions hold. While some assumptions differ between types of test, others are common to all hypothesis tests.

Which of the following statements is a common assumption of hypothesis tests?

# Possible Answers

(x) Sample observations are collected deterministically from the population.

( ) Sample observations are correlated with each other.

( ) Sample observations have no direct relationship with each other.

( ) Sample sizes are greater than thirty observations.

# Testing sample size

In order to conduct a hypothesis test, and be sure that the result is fair, a sample must meet three requirements: it is a random sample of the population; the observations are independent; and there are enough observations. Of these, only the last condition is easily testable with code.

The minimum sample size depends on the type of hypothesis tests you want to perform. Let's test some scenarios on the late_shipments dataset.

late_shipments is available; dplyr is loaded.

# Instructions:

- Using the late_shipments dataset, get counts by the freight_cost_group columns.
- Insert a suitable number to inspect whether the counts are "big enough" for a two sample t-test.



In [ ]:
# Get counts by freight_cost_group
counts <- late_shipments %>%
  count(freight_cost_group)

# See the result
counts

# Inspect whether the counts are big enough
all(counts$n >= 30)

- Using the late_shipments dataset, get counts by the late column.
Insert a suitable number to inspect whether the counts are "big enough" for a one sample proportion test.

In [ ]:
# Get counts by late
counts <- late_shipments %>%
  count(late)

# See the result
counts

# Inspect whether the counts are big enough
all(counts$n >= 10)

- Using the late_shipments dataset, get counts by the vendor_inco_term and freight_cost_group columns.
Insert a suitable number to inspect whether the counts are "big enough" for a chi-square independence test.

In [ ]:
# Count the values of vendor_inco_term and freight_cost_group
counts <- late_shipments %>%
  count(vendor_inco_term, freight_cost_group)

# See the result
counts

# Inspect whether the counts are big enough
all(counts$n >= 5)

- Using the late_shipments dataset, get counts by the shipment_mode column.
Insert a suitable number to inspect whether the counts are "big enough" for an ANOVA test.

In [ ]:
# Count the values of shipment_mode
counts <- late_shipments %>%
  count(shipment_mode)

# See the result
counts

# Inspect whether the counts are big enough
all(counts$n >= 30)

# There is only one test

You've encountered several types of traditional hypothesis test: t-tests, ANOVA tests, proportion tests, and chi-square tests. You may have noticed that there were similarities in the workflow for performing each test.

Allen Downey proposed that all traditional hypothesis tests were special cases of a generic hypothesis test. He called this the "There is Only One Test" framework, and it forms a "grammar of hypothesis tests", analogous to the "grammar of graphics" implemented by ggplot2.

In which situations will the "There is Only One Test" framework provide p-value and decision rule results that are different than a traditional method like prop_test()?

# Possible Answers

(x) The results should always be similar.

( ) When the assumptions for the traditional method are not met.

( ) When the sample size is large enough that the Central Limit Theorem applies.

( ) When groups in the dataset are perfectly balanced.

# Specifying and hypothesizing

In Chapter 3, you ran a two sample proportion test on the proportion of late shipments across freight cost groups. Recall the hypotheses.

Let's compare that traditional approach using prop_test() with a simulation-based infer pipeline.

late_shipments is available; dplyr and infer are loaded.

# Instructions:

Question
Run the proportion test code (previously seen in Chapter 3). Assuming a significance level of alpha = 0.05, what does the evidence suggest?

Possible answers

(x) The p-value is less than or equal to the significance level, so you should reject the null hypothesis that the proportion of late shipments is the same for each freight cost group.

( ) The p-value is less than or equal to the significance level, so you should fail to reject the null hypothesis that the proportion of late shipments is the same for each freight cost group.

( ) The p-value is greater than the significance level, so you should reject the null hypothesis that the proportion of late shipments is the same for each freight cost group.

( ) The p-value is greater than the significance level, so you should fail to reject the null hypothesis that the proportion of late shipments is the same for each freight cost group.

In [ ]:
DM.result = 1

- Using the late_shipments dataset, specify that we are interested in late proportions across freight_cost_group, where "Yes" denotes success.

In [ ]:
# Specify that we are interested in late proportions across freight_cost_groups, where "Yes" denotes success
specified <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  )

# See the result
specified

- Extend the pipeline to declare a null hypothesis that the variables are independent.

In [ ]:
# Extend the pipeline to declare a null hypothesis that the variables are independent
hypothesized <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence")

# See the result
hypothesized

# Generating & calculating

The infer pipeline for hypothesis testing requires four steps to calculate the null distribution: specify, hypothesize, generate, and calculate.

Let's continue the pipeline you began in the previous coding exercise. We'll get a set of differences in proportions that are distributed as though the null hypothesis, that the proportion of late shipments is the same across freight cost groups, is true.

late_shipments is available; dplyr, infer, and ggplot2 are loaded.

# Instructions:

- Extend the infer pipeline to generate two thousand permutation replicates. (Note this will take a few seconds to complete.)



In [ ]:
# Extend the pipeline to generate 2000 permutations
generated <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute")

# See the result
generated

- Complete the infer pipeline for the null distribution by calculating the difference in proportions, setting the order to expensive proportion minus reasonable proportion.

In [ ]:
# Extend the pipeline to calculate the difference in proportions (expensive minus reasonable)
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# See the result
null_distn

- Visualize the null distribution.

In [ ]:
# From previous steps
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Visualize the null distribution
visualize(null_distn)

# Observed statistic and p-value

You now have a null distribution. In order to get a p-value and weigh up the evidence against the null hypothesis, you need to calculate the difference in proportions that is observed in the late_shipments sample.

late_shipments is available; dplyr, infer, and ggplot2 are loaded.

# Instructions:

- Copy, paste, and modify the null distribution pipeline to get the observed statistic.

In [ ]:
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Copy, paste, and modify the pipeline to get the observed statistic
obs_stat <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# See the result
obs_stat

- Visualize the null distribution, adding a vertical line at the observed statistic.

In [ ]:
# From previous steps
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )
obs_stat <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Visualize the null dist'n, adding a vertical line at the observed statistic
visualize(null_distn) +
  geom_vline(aes(xintercept = stat), data = obs_stat)

- Get the p-value from the null distribution and observed statistic, assuming an appropriate direction for the alternative hypothesis.

In [ ]:
# From previous steps
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )
obs_stat <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat, 
  direction = "greater"
)

# See the result
p_value

# Simulation-based t-test

In Chapter 2 you manually performed the steps for a t-test to explore these hypotheses.

: The mean weight of shipments that weren't late is the same as the mean weight of shipments that were late.

: The mean weight of shipments that weren't late is less than the mean weight of shipments that were late.

You can run the test more concisely using infer's t_test().

late_shipments %>% 
  t_test(
    weight_kilograms ~ late,
    order = c("No", "Yes"),
    alternative = "less"
  )
t_test() assumes that the null distribution is normal. We can avoid assumptions by using a simulation-based non-parametric equivalent.

late_shipments is available; dplyr and infer are loaded.

# Instructions:

- Specify weight in kilograms versus whether or not the shipment was late.
- Declare a null hypothesis of independence.
- Generate 1000 permutation replicates.
- Calculate the difference in means, setting the order as "No" minus "Yes".

In [ ]:
# Fill out the null distribution pipeline
null_distn <- late_shipments %>% 
  # Specify weight_kilograms vs. late
  specify(weight_kilograms ~ late) %>% 
  # Declare a null hypothesis of independence
  hypothesize(null = "independence") %>% 
  # Generate 1000 permutation replicates
  generate(reps = 1000, type = "permute") %>% 
  # Calculate the difference in means ("No" minus "Yes")
  calculate(stat = "diff in means", order = c("No", "Yes"))

# See the results
null_distn

- Calculate the difference in means observed in the late_shipments dataset.

In [ ]:
# From previous step
null_distn <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 1000, type = "permute") %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))

# Calculate the observed difference in means
obs_stat <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))

# See the result
obs_stat

- Get the p-value from the null distribution and the observed difference in means, setting an appropriate direction.

In [ ]:
# From previous steps
null_distn <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 1000, type = "permute") %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))
obs_stat <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat,
  direction = "less"
)

# See the result
p_value

# Rank sum tests

Another class of non-parametric hypothesis tests are called rank sum tests. Ranks are the positions of numeric values from smallest to largest. Think of them as positions in running events: whoever has the fastest (smallest) time is rank 1, second fastest is rank 2, and so on.

By calculating on the ranks of data instead of the actual values, you can avoid making assumptions about the distribution of the test statistic. It's most robust in the same way that a median is more robust than a mean.

Two commonly used rank-based tests are the Wilcoxon-Mann-Whitney test, which is like a non-parametric t-test, and the Kruskal-Wallis test, which is like a non-parametric ANOVA.

late_shipments is available.

# Instructions:

- Using the late_shipments dataset, run a Wilcoxon-Mann-Whitney test on the weight in kilograms versus whether or not the shipment was late.

In [ ]:
# Run a Wilcoxon-Mann-Whitney test on weight_kilograms vs. late
test_results <- wilcox.test(
  weight_kilograms ~ late, 
  data = late_shipments
)

# See the result
test_results

- Using the late_shipments dataset, run a Kruskal-Wallace test on the weight in kilograms versus the shipment mode.

In [ ]:
# Run a Kruskal-Wallace test on weight_kilograms vs. shipment_mode
test_results <- kruskal.test(
  weight_kilograms ~ shipment_mode, 
  data = late_shipments
)

# See the result
test_results